In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Pre-requisites:**

In [ ]:
!curl https://raw.githubusercontent.com/automl/auto-sklearn/master/requirements.txt | xargs -n 1 -L 1 pip install
!pip install auto-sklearn

In [ ]:
!pip install dtreeviz
from dtreeviz.trees import dtreeviz

In [ ]:

!pip install -U rfpimp
!pip install -U imbalanced-learn
%matplotlib inline
#importing libraries
import numpy as np
import scipy.stats
import scipy.special
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from matplotlib import cm
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import log_loss
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import make_pipeline, make_union, Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import ParameterGrid
from keras.models import Sequential
from keras.models import Model as KerasModel
from keras.layers import Input, Dense, Activation, Reshape
from keras.layers import Concatenate
from keras.layers.embeddings import Embedding
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.inspection import permutation_importance
import pickle
import csv
import collections
from rfpimp import *
from rfpimp import plot_corr_heatmap
from datetime import datetime
from sklearn import preprocessing
from keras.callbacks import ModelCheckpoint
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report
import seaborn as sns
import imblearn
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import make_scorer
from sklearn.metrics import plot_confusion_matrix
import math
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.layers import Input, LSTM, Dense, Concatenate,GRU,Dropout,LeakyReLU
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.io import FixedLenFeature
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import LabelEncoder,OneHotEncoder, normalize
import tensorflow as tf

# Datasets:

In [ ]:
training=pd.read_csv("/kaggle/input/higgs-boson/training.zip")
training.head()

In [ ]:
testing=pd.read_csv("/kaggle/input/higgs-boson/test.zip")
testing.set_index('EventId',inplace=True)
testing.head()

*Here, we have the two datasets of the Higg Boson Challenge of 2014 which had the data for categorising the events as either a signal event or a background event. The accuracy of the classification model is to be calculated using the approximate median significance. The weights of each of the cases are also given.*

In [ ]:
print(training.shape,testing.shape)

In [ ]:
train_dataX=training.drop(["Weight","Label"],axis=1)
train_dataX.set_index('EventId',inplace=True)
train_dataY=training['Label']


In [ ]:
train_dataY=pd.factorize(train_dataY)[0]

In [ ]:
from IPython.display import display, HTML

def pretty_print(df):
    return display( HTML( df.to_html().replace("\\n","<br>") ) )
def tbl_report(tbl, cols=None, card=10):
    print("Table Shape", tbl.shape)
    dtypes = tbl.dtypes
    nulls = []
    uniques = []
    numuniques = []
    vcs = []
    for col in dtypes.index:
        n = tbl[col].isnull().sum()
        nulls.append(n)
        strdtcol = str(dtypes[col])
        #if strdtcol == 'object' or strdtcol[0:3] == 'int' or strdtcol[0:3] == 'int':
        #print(strdtcol)
        uniqs = tbl[col].unique()
        uniquenums = uniqs.shape[0]
        if uniquenums < card: # low cardinality
            valcounts = pd.value_counts(tbl[col], dropna=False)
            vc = "\n".join(["{}:{}".format(k,v) for k, v in valcounts.items()])
        else:
            vc='HC' # high cardinality
        uniques.append(uniqs)
        numuniques.append(uniquenums)
        vcs.append(vc)
    nullseries = pd.Series(nulls, index=dtypes.index)
    uniqueseries = pd.Series(uniques, index=dtypes.index)
    numuniqueseries = pd.Series(numuniques, index=dtypes.index)
    vcseries = pd.Series(vcs, index=dtypes.index)
    df = pd.concat([dtypes, nullseries, uniqueseries, numuniqueseries, vcseries], axis=1)
    df.columns = ['dtype', 'nulls', 'uniques', 'num_uniques', 'value_counts']
    if cols:
        return pretty_print(df[cols])
    return pretty_print(df)

**We will have a look at our training dataset before we run any EDA,Feature Engineering or Modelling**

In [ ]:
tbl_report(train_dataX)

Our dataset has one categorical variable and rest all continuous variable.

In [ ]:
cat_vars=['PRI_jet_num']
cont_vars=np.array(train_dataX.drop('PRI_jet_num',axis=1).columns)

In [ ]:
cont_vars

# EDA:

In [ ]:
pd.DataFrame(collections.Counter(train_dataY),index=['0','1']).iloc[0]



*This shows that our dataset is highly imbalanced and we will be needing to do certain upsampling and downsampling in our data.*

In [ ]:
fig,axes=plt.subplots(figsize=(10,8))
print(training['Label'].value_counts())
sns.barplot(x = training['Label'].value_counts().index, y = training['Label'].value_counts().values)
plt.title('Label counts')
plt.show()


*As we can see this is an unbalanced dataset, with more number of background events, characterised by 1 and less number of signal events characterised by 0*

In [ ]:
ncols = 2
nrows = math.ceil(len(cont_vars)/ncols)
fig, axen = plt.subplots(nrows, ncols, figsize = (12, nrows*4))
for v, ax in zip(cont_vars, axen.ravel()):
    sns.histplot(train_dataX[v], ax=ax)
    ax.set_xscale('log')

In [ ]:
#viz = plot_corr_heatmap(train_dataX, figsize=(11,10))
#viz.save('corrheatmap.svg')
#viz

In [ ]:
sns.pairplot(data=train_dataX[cont_vars])

In [ ]:
D = feature_dependence_matrix(train_dataX)
viz = plot_dependence_heatmap(D, figsize=(11,10))
viz

*Here we have got hold of the multiple dependencies of the different variables. We will eliminate the collinearity of the variables while checking for the fearure importances post modelling.*

# Models:

#### Helper Function:

In [ ]:
def cv_optimize(clf, parameters, X, y, n_jobs=1, n_folds=5, score_func=None):
    if score_func:
        print("SCORE FUNC", score_func)
        gs = GridSearchCV(clf, param_grid=parameters, cv=n_folds, n_jobs=n_jobs, scoring=score_func)
    else:
        gs = GridSearchCV(clf, param_grid=parameters, n_jobs=n_jobs, cv=n_folds)
    gs.fit(X, y)
    print("BEST", gs.best_params_, gs.best_score_)
    best = gs.best_estimator_
    return best
def do_classify(clf, parameters, indf,y,score_func, n_folds=5, n_jobs=1):
    X=indf
    y=y
    Xtrain,Xtest,ytrain,ytest=train_test_split(X,y,train_size=0.8,random_state=2017)
    clf = cv_optimize(clf, parameters, Xtrain, ytrain, n_jobs=n_jobs, n_folds=n_folds, score_func=score_func)
    clf=clf.fit(Xtrain, ytrain)
    training_accuracy = clf.score(Xtrain, ytrain)
    test_accuracy = clf.score(Xtest, ytest)
    print("############# based on standard predict ################")
    print("Accuracy on training data: %0.2f" % (training_accuracy))
    print("Accuracy on test data:     %0.2f" % (test_accuracy))
    print(confusion_matrix(ytest, clf.predict(Xtest)))
    print("########################################################")
    plot_confusion_matrix(clf,Xtest,ytest,cmap="Blues")
    return clf, Xtrain, ytrain, Xtest,ytest

### Baseline Model using LogisticRegression:

In [ ]:
# set up standardization
ss = StandardScaler()
# oe hot encoding
oh = OneHotEncoder()
# continuous variables need to be standardized
cont_pipe = Pipeline([("scale", ss)])
# categorical variables need to be one hot encoded
cat_pipe = Pipeline([('onehot', oh)])
# combine both into a transformer
transformers = [('cont', cont_pipe, cont_vars), ('cat', cat_pipe, cat_vars)]
# apply transformer to relevant columns. Nothing will be done for the rest
ct = ColumnTransformer(transformers=transformers, remainder="passthrough")
# create a pipeline so that we are not leaking data from validation to train in the individual folds
pipe = Pipeline(steps=[('ct', ct), ('model', LogisticRegression(max_iter=10000, penalty='l2'))])
# in paramgrid we dont use C but use model__C corresponding to the name in the pipeline
paramgrid = dict(model__C=[1000, 100, 10, 1, 0.1, 0.01, 0.001])

lr,Xtrain,ytrain,Xtest,ytest=do_classify(pipe, paramgrid,train_dataX, 
                              train_dataY, 
                              score_func='roc_auc')

In [ ]:
print(classification_report(ytest,lr.predict(Xtest)))

*The imbalance is causing the accuracy issue for predicting the the signal event. Hence, we will balance the dataset by upsampling and downsampling.*

### Upsampling and DownSampling:

In [ ]:
over = SMOTE(sampling_strategy='auto', k_neighbors=2)
under = RandomUnderSampler(sampling_strategy='auto')
steps = [('o', over), ('u', under)]
pipeline = imblearn.pipeline.Pipeline(steps=steps)
X, y = pipeline.fit_resample(train_dataX,train_dataY)
counter = collections.Counter(y)
print(counter)

### Baseline Model using Decision Tree Classifier:

In [ ]:
!pip install dtreeviz
from dtreeviz.trees import dtreeviz

In [ ]:
clf=DecisionTreeClassifier(random_state=2017)
parameters={'max_depth':range(1,9),'min_samples_leaf':range(3,5),'criterion':['gini']}
clf, Xtrain, ytrain, Xtest,ytest=do_classify(clf, parameters,train_dataX,train_dataY,'roc_auc',n_folds=5,n_jobs=-1)

In [ ]:
print(classification_report(ytest,clf.predict(Xtest)))

In [ ]:
colors = [None,  # 0 classes
          None,  # 1 class
          ['#FFF4E5','#D2E3EF'],# 2 classes
           ]
vizA = dtreeviz(clf, train_dataX,train_dataY,
               feature_names = train_dataX.columns,
               target_name = 'Label', class_names= ['No','Yes']
              ,orientation = 'TD',
               colors={'classes':colors},
               label_fontsize=14,
               ticks_fontsize=10,
               )
vizA

In [ ]:
vizA.save('tree.svg')

**Although the dataset is imbalanced,the decision tree seems to do a decent job wrt to handling the imbalance**

In [ ]:

def p_importance(model, cols, fi, fistd = 0):
    return pd.DataFrame({'features':cols, 'importance':fi, 'importance_std': fistd}
                       ).sort_values('importance', ascending=False)
dimp=permutation_importance(clf,Xtest,ytest)
ddf=p_importance(clf,list(train_dataX.columns),dimp['importances_mean'],dimp['importances_std']).iloc[:10]
fig,ax=plt.subplots(figsize=(17,10))
sns.barplot(data=ddf,x='features',y='importance',label='Decision_importances',ax=ax)
plt.xticks(rotation='45')
plt.title("Bar plot of Importances for Decision Tree Model");

In [ ]:
D = D['Dependence'].sort_values(ascending=False)
D

**we see in the dependences and importance plots, the most dependent variables are the least important amongst the top 10 as well.**

In [ ]:
##taking only those features which are important
features=['DER_mass_MMC','DER_mass_transverse_met_lep','DER_mass_vis','DER_deltar_tau_lep','PRI_tau_pt','DER_met_phi_centrality','DER_pt_h','PRI_met']

In [ ]:
clf=DecisionTreeClassifier(random_state=2017)
parameters={'max_depth':range(1,9),'min_samples_leaf':range(3,5),'criterion':['gini']}
clf, Xtrain, ytrain, Xtest,ytest=do_classify(clf, parameters,X[features],y,'roc_auc',n_folds=5,n_jobs=-1)

In [ ]:
print(classification_report(ytest,clf.predict(Xtest)))

In [ ]:
colors = [None,  # 0 classes
          None,  # 1 class
          ['#FFF4E5','#D2E3EF'],# 2 classes
           ]
vizb = dtreeviz(clf, Xtest[features],ytest,
               feature_names = X[features].columns,
               target_name = 'Label', class_names= ['No','Yes']
              ,orientation = 'TD',
               colors={'classes':colors},
               label_fontsize=14,
               ticks_fontsize=10,
               )
vizb

In [ ]:
vizb.save('tree2.svg')

### Random Forest Classifier:

In [ ]:
clf2=RandomForestClassifier(random_state=2017)
parameters={"n_estimators":[400],'max_features':[3],'max_depth':[5]}
clf2, Xtrain, ytrain, Xtest, ytest  = do_classify(clf2, 
   parameters,train_dataX[features],train_dataY,"roc_auc", n_folds=5, n_jobs=1)

In [ ]:
print(classification_report(ytest,clf2.predict(Xtest)))

*We do the same now with the balanced dataset.*

In [ ]:

rimp=permutation_importance(clf2,Xtest,ytest)
rdf=p_importance(clf2,features,rimp['importances_mean'],rimp['importances_std'])
fig,ax=plt.subplots(figsize=(17,10))
sns.barplot(data=rdf,x='features',y='importance',label='RandomForest_importances',ax=ax)
plt.xticks(rotation='45')
plt.title("Bar plot of Importances for Random Forest Model");

Since, we know this is an imbalanced dataset, we will try out an upsampling downsampling for this purpose and run our model on the balanced dataset, to see how well our model does and if the upsampling helps.

In [ ]:
clf2=RandomForestClassifier(random_state=2017)
parameters={"n_estimators":[400],'max_features':[3],'max_depth':[5]}
clf2, Xtrain, ytrain, Xtest, ytest  = do_classify(clf2, 
   parameters,X[features],y,"roc_auc", n_folds=5, n_jobs=1)


In [ ]:
print(classification_report(ytest,clf2.predict(Xtest)))

### Boosting:

### Boosting without Balancing Dataset:

In [ ]:
Xtrain,Xtest,ytrain,ytest=train_test_split(train_dataX[features],train_dataY,train_size=0.8)
Xtrain.shape,ytrain.shape
dtrainimb=xgb.DMatrix(Xtrain,label=ytrain)

clf3 = xgb.train(xgb_pars, dtrainimb, 500,
                  maximize=False, verbose_eval=15) 


In [ ]:
dvalid = xgb.DMatrix(Xtest, label=ytest)
y_predimb=clf3.predict(dvalid)
y_predimb_=[1 if y>0.5 else 0 for y in y_predimb]
print(classification_report(ytest,y_predimb_))

### Boosting with the Balanced Data:

In [ ]:
Xtrain,Xtest,ytrain,ytest=train_test_split(X[features],y,train_size=0.8)
Xtrain.shape,ytrain.shape

In [ ]:
dtrain = xgb.DMatrix(Xtrain, label=ytrain)
dvalid = xgb.DMatrix(Xtest, label=ytest)

xgb_pars = {'min_child_weight': 100, 'eta': 0.04, 'colsample_bytree': 0.8, 'max_depth': 100,
             'subsample': 0.75, 'lambda': 2, 'nthread': -1, 'booster' : 'gbtree', 'silent': 1, 'gamma' : 0,
             'eval_metric': 'mae', 'objective': 'reg:linear'}    

clf3 = xgb.train(xgb_pars, dtrain, 500,
                  maximize=False, verbose_eval=15) 

In [ ]:
y_pred=clf3.predict(dvalid)

In [ ]:
y_pred_=[1 if y>0.5 else 0 for y in y_pred]

In [ ]:
print(classification_report(ytest,y_pred_))

*Without the balancing of the dataset, the model runs poorly and hence the balancing.*

# Comparing the ROC curves:

In [ ]:

def make_roc(name, clf, ytest, xtest, ax=None, labe=5,  proba=True, skip=0, initial = False):
    if not ax:
        ax=plt.gca()
    if proba:
        fpr, tpr, thresholds=roc_curve(ytest, clf.predict_proba(xtest)[:,1])
    else:
        fpr, tpr, thresholds=roc_curve(ytest, clf.decision_function(xtest))
    roc_auc = auc(fpr, tpr)
    if skip:
        l=fpr.shape[0]
        ax.plot(fpr[0:l:skip], tpr[0:l:skip], '.-', lw=2, alpha=0.4, label='ROC curve for %s (area = %0.2f)' % (name, roc_auc))
    else:
        ax.plot(fpr, tpr, '.-', lw=2, alpha=0.4, label='ROC curve for %s (area = %0.2f)' % (name, roc_auc))
    label_kwargs = {}
    label_kwargs['bbox'] = dict(
        boxstyle='round,pad=0.3', alpha=0.2,
    )
    for k in range(0, fpr.shape[0],labe):
        #from https://gist.github.com/podshumok/c1d1c9394335d86255b8
        threshold = str(np.round(thresholds[k], 2))
        ax.annotate(threshold, (fpr[k], tpr[k]), **label_kwargs)
    if initial:
        ax.plot([0, 1], [0, 1], 'k--')
        ax.set_xlim([0.0, 1.0])
        ax.set_ylim([0.0, 1.05])
        ax.set_xlabel('False Positive Rate')
        ax.set_ylabel('True Positive Rate')
        ax.set_title('ROC')
    ax.legend(loc="lower right")
    return ax

In [ ]:
fig,ax=plt.subplots(figsize=(25,25))
xtrain,xtest,ytr,yte=train_test_split(train_dataX,train_dataY,train_size=0.8)
make_roc('logistic', lr,yte ,xtest, ax=ax, labe=10,  proba=True, skip=16300, initial = False)



In [ ]:
fig,ax=plt.subplots(figsize=(25,25))
make_roc('random', clf2,ytest ,Xtest, ax=ax, labe=10,  proba=True, skip=16300, initial = False)

In [ ]:
fig,ax=plt.subplots(figsize=(25,25))
make_roc('decisiontree', clf,ytest ,Xtest, ax=ax, labe=10,  proba=True, skip=0, initial = False)

**The ** 

### Recurrent Neural Networks 

*In a recurrent neural network we store the output activations from one or more of the layers of the network. Often these are hidden later activations. Then, the next time we feed an input example to the network, we include the previously-stored outputs as additional inputs. You can think of the additional inputs as being concatenated to the end of the “normal” inputs to the previous layer. For example, if a hidden layer had 10 regular input nodes and 128 hidden nodes in the layer, then it would actually have 138 total inputs (assuming you are feeding the layer’s outputs into itself à la Elman) rather than into another layer). Of course, the very first time you try to compute the output of the network you’ll need to fill in those extra 128 inputs with 0s or something.image.png
RNNs are quite powerful, they suffer from Vanishing gradient problem which hinders them from using long term information, like they are good for storing memory 3-4 instances of past iterations but larger number of instances don't provide good results so we don't just use regular RNNs. Instead, we use a better variation of RNNs: Long Short Term Networks(LSTM).*

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE

In [ ]:
train_df=training.drop('Weight',axis=1)

In [ ]:
enc = LabelEncoder()

train_df['Label'] = enc.fit_transform(train_df['Label'])
train_df.head()

In [ ]:
train_df.set_index(['EventId'],inplace = True)


In [ ]:
testing.shape,train_df.shape

In [ ]:
X = train_df.drop(['Label'], axis=1)
y = train_df['Label'].values
X = normalize(X)
test_df_nor = normalize(testing)

In [ ]:
#split to maintain the imbalance
splitter=StratifiedShuffleSplit(n_splits=1,random_state=12)

for train,test in splitter.split(X,y):     
    X_train = X[train]
    y_train = y[train]
    X_test = X[test]
    y_test = y[test]

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))

In [ ]:
import os
import csv
import math


def create_solution_dictionary(solution):
    """ Read solution file, return a dictionary with key EventId and value (weight,label).
    Solution file headers: EventId, Label, Weight """
    
    solnDict = {}
    with open(solution, 'rb') as f:
        soln = csv.reader(f)
        soln.next() # header
        for row in soln:
            if row[0] not in solnDict:
                solnDict[row[0]] = (row[1], row[2])
    return solnDict
def check_submission(submission, Nelements):
    """ Check that submission RankOrder column is correct:
        1. All numbers are in [1,NTestSet]
        2. All numbers are unqiue
    """
    rankOrderSet = set()    
    with open(submission, 'rb') as f:
        sub = csv.reader(f)
        sub.next() # header
        for row in sub:
            rankOrderSet.add(row[1])
            
    if len(rankOrderSet) != Nelements:
        print ('RankOrder column must contain unique values')
        exit()
    elif rankOrderSet.isdisjoint(set(xrange(1,Nelements+1))) == False:
        print ('RankOrder column must contain all numbers from [1..NTestSset]')
        exit()
    else:
        return True
    def AMS(s, b):
        """ Approximate Median Significance defined as:
            AMS = sqrt(
                    2 { (s + b + b_r) log[1 + (s/(b+b_r))] - s}
                  )        
        where b_r = 10, b = background, s = signal, log is natural logarithm """

        br = 10.0
        radicand = 2 *( (s+b+br) * math.log (1.0 + s/(b+br)) -s)
        if radicand < 0:
            print ('radicand is negative. Exiting')
            exit()
        else:
            return math.sqrt(radicand)
    def AMS_metric(solution, submission):
        """  Prints the AMS metric value to screen.
        Solution File header: EventId, Class, Weight
        Submission File header: EventId, RankOrder, Class
        """

        numEvents = 550000 # number of events = size of test set

        # solutionDict: key=eventId, value=(label, class)
        solutionDict = create_solution_dictionary(solution)

        signal = 0.0
        background = 0.0
        if check_submission(submission, numEvents):
            with open(submission, 'rb') as f:
                sub = csv.reader(f)
                sub.next() # header row
                for row in sub:
                    if row[2] == 's': # only events predicted to be signal are scored
                        if solutionDict[row[0]][0] == 's':
                            signal += float(solutionDict[row[0]][1])
                        elif solutionDict[row[0]][0] == 'b':
                            background += float(solutionDict[row[0]][1])

            print ('signal = {0}, background = {1}'.format(signal, background))
            print ('AMS = ' + str(AMS(signal, background)))


        if __name__ == "__main__":

            # enter path and file names here    
            path = ""
            solutionFile = ""
            submissionFile = ""

In [ ]:
def build_rnn_model(train_x,train_y,test_x,test_y):
    inp = Input(shape=(train_x.shape[1],train_x.shape[2]))
    rnn_1st_model = LSTM(units=60, return_sequences=True,recurrent_dropout=0.1)(inp)
    rnn_2nd_model = LSTM(units=60,recurrent_dropout=0.1)(rnn_1st_model)
    dense_layer = Dense(128)(rnn_2nd_model)
    drop_out = Dropout(0.2)(dense_layer)
    output = Dense(1, activation= LeakyReLU(alpha=0.1),name="class")(drop_out)
    model = Model(inp, output)
    callbacks = [ReduceLROnPlateau(monitor='val_loss', patience=4, verbose=1, factor=0.6),
                 EarlyStopping(monitor='val_loss', patience=20),
                 ModelCheckpoint(filepath='best_model_LSTM.h5', monitor='val_loss', save_best_only=True)]
    model.summary()
    model.compile(loss=[tf.keras.losses.MeanSquaredLogarithmicError(),
                        tf.keras.losses.MeanSquaredLogarithmicError()], optimizer="adam")
    history = model.fit(train_x, train_y, 
          epochs = 40, 
          batch_size = 100, 
          validation_data=(test_x,  test_y), 
          callbacks=callbacks)
    return history,model

In [ ]:
def plot_Loss(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Loss over epochs')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='best')
    plt.show()

In [ ]:
def Save_Result_To_Csv(model,model_name,csv_file):
    test_pre = np.reshape(test_df_nor, (test_df_nor.shape[0],test_df_nor.shape[1],1))
    model.load_weights(model_name)
    prediction = model.predict(test_pre)
    prediction =  np.where(prediction > 0.5, 1, 0)
    prediction = pd.Series(prediction[:,0])
    sub = pd.read_csv('../input/higgs-boson/random_submission.zip')
    test_predict = pd.DataFrame({"EventId":sub['EventId'],"RankOrder":sub['RankOrder'],"Class":prediction})
    test_predict['Class'] = test_predict['Class'].replace(1,'s')
    test_predict['Class'] = test_predict['Class'].replace(0,'b')
    test_predict.to_csv(csv_file,index=False)

In [ ]:
history_RNN,Rnn_model = build_rnn_model(X_train,y_train,X_test,y_test)

In [ ]:
plot_Loss(history_RNN)

In [ ]:
Save_Result_To_Csv(Rnn_model,"./best_model_LSTM.h5","submission_Rnn.csv")

In [ ]:
sub=pd.read_csv("./submission_Rnn.csv")
sub.head()
